# Same differential evolution implementation with gaussian noise on mass constraint

In [ ]:
%reset
import numpy as np
import csv
from scipy.optimize import *
import sys
import matplotlib.pyplot as plt
np.set_printoptions(threshold=sys.maxsize)
import datetime
from scipy.stats import skewnorm

In [ ]:
#import cell

#science
science_array = []

#take data from experiments csv and convert it to an array
with open('Found_Experiments.csv', 'rt') as f:
    array_y = csv.reader(f, skipinitialspace=True, quotechar="'")
    for line in array_y:
        science_array.append(line)
#print(science_array)


#take data from scores science csv and convert it to an array
objectives_score = []
with open('Objective_scores.csv', 'rt') as f:
    array_x = csv.reader(f, skipinitialspace=True, quotechar="'")
    for line in array_x:
        objectives_score.append(line)


In [ ]:
#data processing cell

#Parse objective data
science_array_array_mod = science_array[1:]                                         #cut off title score
science_mass_array      = np.asarray([i[-1] for i in science_array_array_mod],float)      #Masses of experiments 0 indexed
science_power_array     = np.asarray([i[-2] for i in science_array_array_mod],float)      #power consumption of experiments 0 indexed
objective_array_temp    = [np.array(i[1:-2]) for i in science_array_array_mod]  
objective_array         = []
for i in objective_array_temp:
    arr = []
    for j in i:
        s = j
        y = ""
        for char in s:
            if char.isdigit():
                y += char
        num = int(y)
        arr.append(num)
    
    objective_array.append(arr)
objective           = np.array(objectives_score[1:])
objective           = np.asarray(objective[0:, 1:],float)
objective_matrix    = np.zeros((len(objective_array),len(objective)))
for i in range(len(objective_matrix)):
    for j in objective_array[i]:
        objective_matrix[i][j-1] = 1

objective_matrix = np.transpose(objective_matrix) #matrix of shortfalls where the column are a tech and the value (0 or 1) represents of that index of a shortfall is completed

objective_Understanding = objective[0:, 1]
objective_Humans        = objective[0:, 2]

beta = 185 #kg/kW Power to mass ratio
max_mass    = 40000 #kg
min_mass    = 15000 #kg
mass_ratio  =  0.3 #kg

In [ ]:
#random guassian
def sample_skewed_gaussian(peak, skew, scale=2, size=1, domain=(0, 60000)):
    loc = peak
    samples = []
    max_attempts = 10000 
    x = np.linspace(*domain, 1000)
    pdf_vals = skewnorm.pdf(x, a=skew, loc=loc, scale=scale)
    pdf_max = np.max(pdf_vals)
    attempts = 0
    while len(samples) < size and attempts < max_attempts:
        x_try = np.random.uniform(*domain)
        y_try = np.random.uniform(0, pdf_max)
        pdf_val = skewnorm.pdf(x_try, a=skew, loc=loc, scale=scale)
        if y_try <= pdf_val:
            samples.append(x_try)
        attempts += 1
    if len(samples) < size:
        raise RuntimeError("Sampling failed — try increasing max_attempts or adjusting parameters.")
    return np.array(samples) if size > 1 else samples[0]

In [ ]:
M = []
s = []
m = []
for j in range(0,100):

    ML = sample_skewed_gaussian(15000,skew=0,scale=2500)
    MU = sample_skewed_gaussian(40000,skew=0,scale=2500)
    
    max_mass    = MU #kg
    min_mass    = ML #kg
    mass_ratio  =  0.3 #kg
    M.append([ML,MU])
    print(M)

    def score_science(mission, w1=4,w2=7,w3=5,check_mass=True):
        mass_max_science = max_mass*mass_ratio #in kg
        mass_min_science = min_mass*mass_ratio #in kg

        #Mass check
        if check_mass and (science_mass_array.dot(mission) + beta*science_power_array.dot(mission) <= mass_min_science or science_mass_array.dot(mission) + beta*science_power_array.dot(mission) > mass_max_science):
            return abs(science_mass_array.dot(mission) + beta*science_power_array.dot(mission))

        #normlizers
        max_number      = 46       #normalize number score 0-1
        max_understand  = 5        #normlize understanding score 0-1
        max_human       = 5        #normlize human score 0-1  

        #normlize weights to sum to 10
        W  = w1+w2+w3
        w1 = 10*w1/W
        w2 = 10*w2/W
        w3 = 10*w3/W

        mission                     = mission.astype(int) #Convete to ints
        completed                   = np.clip(objective_matrix @ mission, a_min=None, a_max=1) #A objective is either completed or not
        completed_number_score      = np.linalg.norm(completed)**2                             #Number of objective completed
        normalized_completed        = completed/completed_number_score                         #Normlizes completed array to weight each score

        completed_understanding_score   = objective_Understanding @ normalized_completed       #Generates the understanding score
        completed_human_score           = objective_Humans   @ normalized_completed            #Generates the human score

        #score calculation
        score = w1*completed_number_score/max_number + w2*completed_understanding_score/max_understand + w3*completed_human_score/max_human
        return -score

    scores_per_sci = np.zeros(len(science_array)-1)
    for i in range(len(scores_per_sci)):
        input = np.zeros(len(science_array)-1)
        input[i] = 1
        scores_per_sci[i] = score_science(input,check_mass=False)
    indcies_tech = np.argsort(scores_per_sci)
    scores_per_tech_sort = np.sort(scores_per_sci) 

    #rint(scores_per_tech_sort)
    #print(indcies_tech)
    mass = 0
    bring_array_science = np.zeros(len(science_array)-1)
    for i in range(len(indcies_tech)):
        if mass < 0.5*(max_mass+min_mass)*(mass_ratio):
            bring_array_science[indcies_tech[i]] = 1
            mass = science_mass_array.dot(bring_array_science) + beta*science_power_array.dot(bring_array_science)

    print(bring_array_science)
    print(score_science(bring_array_science))

    bounds_science  = Bounds(lb=np.zeros(len(objective_array)), ub=np.ones(len(objective_array)))                                                       #we either bring or we dont
    seed            = np.random.randint(9999999)
    res_science     = differential_evolution(func=score_science, bounds=bounds_science, x0=bring_array_science, integrality=np.ones(len(science_array)-1),updating='deferred',tol=0.01,init='sobol',popsize=16,recombination=0.05,mutation=(0.7,1.7),maxiter=1000,seed=seed)
    s.append(seed)
    m.append(res_science)


In [ ]:
pops = []
for i in m:
    pops.append(i['x'].tolist())

In [ ]:
now = datetime.datetime.now()
formatted_time = now.strftime("%d-%m-%Y_%H_%M_%S")
import json
data = {
    'seeds' : s,
    'populations' : pops,
    'mass' : M
}
with open('100_guassian_std_0.5_sci_mass' + formatted_time + '.json', 'w') as f:
    json.dump(data, f, indent=4)  

In [ ]:
for i in m:
    print(i)

In [ ]:
outputs_sci = []
count = 0
for i in m:
    if i['success']:
        outputs_sci.append(i['x'])
        count += 1

result = []
with open('Experiments_to_Bring_Rebal-p32_r1_m7-1723-10-2024_11_18_51.csv', 'rt') as f:
    array_x = csv.reader(f, skipinitialspace=True, quotechar="'")
    for line in array_x:
        result.append(line)
result = result[1:]
objectivs_names_temp = []
for i in result:
    objectivs_names_temp.append(i[1])
objectivs_names_temp = objectivs_names_temp[0:-1]

highlight_indices = objectivs_names_temp

percentages = np.mean(outputs_sci, axis=0) * 100
sorted_indices = np.argsort(-percentages)
sorted_percentages = percentages[sorted_indices]
indices = [row[0] for row in science_array_array_mod]
indices = [indices[i] for i in sorted_indices]
plt.figure(figsize=(15,5))
plt.bar(indices, sorted_percentages, color='grey')
plt.ylabel('Occurrence Percentage')
plt.title(f'Common Experiments over {count} trials')
plt.xticks(indices, rotation=90, fontsize=7)

ax = plt.gca()
for tick_label in ax.get_xticklabels():
    if tick_label.get_text() in highlight_indices:
        tick_label.set_color('red')

plt.show()

In [ ]:
def skewed_gaussian(x, peak, skew, scale=2):
    loc = peak 
    raw_pdf = skewnorm.pdf(x, a=skew, loc=loc, scale=scale)
    raw_pdf /= np.trapezoid(raw_pdf, x)
    return raw_pdf

x_vals = np.linspace(0, 10, 1000)
peaks = [4,5,7]
skew = 0  
scale = 0.5
for i in range(0,len(peaks)):
    pdf_vals = skewed_gaussian(x_vals, peak=peaks[i], skew=skew,scale=scale)
    plt.plot(x_vals, pdf_vals, label=f'pdf for w{i+1}')
plt.title('Gaussian Distribution for each weight')
plt.xlabel('Value')
plt.ylabel('Probability Density')
plt.grid(True)
plt.legend()
plt.show()
